In [1]:
import json
with open(file="../../echo-chamber/data/mastodon.trump.json") as f:
    data = json.load(f)

In [2]:
import re
from bs4 import BeautifulSoup
md = []
for post in data:
    if not post['in_reply_to_id']:
        content = post["content"]
        # Parse the HTML content with BeautifulSoup
        soup = BeautifulSoup(content, "html.parser")
    
        # Remove all 'a' tags entirely
        for a_tag in soup.find_all("a"):
            a_tag.unwrap()
    
        # Get the plain text
        plain_text = soup.get_text(separator=" ")
    
        # Regular expression to match and remove URLs
        cleaned_content = re.sub(
            r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F])|\s)+",
            "",
            plain_text,
        )
    
        # Remove excess spaces left by removing the link
        cleaned_content = re.sub(r"\s+", " ", cleaned_content).strip()
        md.append([post['id'],cleaned_content])

/tmp/ipykernel_3423980/1337830669.py:8: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(content, "html.parser")


In [3]:
import pandas as pd
md_df = pd.DataFrame(md, columns=['id','post'])

In [4]:
with open(file="../../echo-chamber/data/bsky.trump.json") as f:
    data = json.load(f)


In [5]:
bsky = []
for post in data:
    if not post['record'].get('reply',None):
        if post['record'].get('text',None):
            bsky.append([post['_id'],post['record']['text']])

In [6]:
import pandas as pd
bsky_df = pd.DataFrame(bsky, columns=['id','post'])

In [7]:
bsky_df.shape

(686010, 2)

In [8]:
with open(file="../../echo-chamber/data/truthsocial.trump.json") as f:
    data = json.load(f)


In [9]:
import re
from bs4 import BeautifulSoup
ts = []
for post in data:
    if post['in_reply_to_id'] or post['quote_id']:
        continue
    else:
        content = post["content"]
        # Parse the HTML content with BeautifulSoup
        soup = BeautifulSoup(content, "html.parser")
    
        # Remove all 'a' tags entirely
        for a_tag in soup.find_all("a"):
            a_tag.unwrap()
    
        # Get the plain text
        plain_text = soup.get_text(separator=" ")
    
        # Regular expression to match and remove URLs
        cleaned_content = re.sub(
            r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F])|\s)+",
            "",
            plain_text,
        )
    
        # Remove excess spaces left by removing the link
        cleaned_content = re.sub(r"\s+", " ", cleaned_content).strip()
        if cleaned_content:
            ts.append([post['id'],cleaned_content])

In [10]:
len(ts)


52129

In [11]:
ts_df = pd.DataFrame(ts, columns=['id','post'])

In [13]:
sum(bsky_df['post'] == '')

0

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Step 1: Initialize the SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2', )  # You can use other SBERT models as well

# Step 2: Define your two lists
list1 = list(bsky_df['post'])
list2 = list(ts_df['post'])

# Step 3: Compute embeddings for both lists
embeddings1 = model.encode(list1, convert_to_tensor=True, show_progress_bar=True, batch_size=512)
embeddings2 = model.encode(list2, convert_to_tensor=True, show_progress_bar=True, batch_size=512)



Batches:   0%|          | 0/1340 [00:00<?, ?it/s]

Batches:   0%|          | 0/102 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 133.22 GiB. GPU 0 has a total capacity of 79.10 GiB of which 69.69 GiB is free. Process 3388522 has 6.36 GiB memory in use. Including non-PyTorch memory, this process has 3.02 GiB memory in use. Of the allocated memory 2.23 GiB is allocated by PyTorch, and 218.83 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [16]:

# Step 4: Compute cosine similarity
cosine_sim = util.cos_sim(embeddings1.cpu(), embeddings2.cpu())

In [26]:
similarity_matrix = cosine_sim
# Set threshold for similarity
threshold = 0.75

# Filter indices where similarity > threshold
row, col = np.where(similarity_matrix > threshold)
weights = similarity_matrix[row, col]



In [27]:
from scipy.sparse import coo_matrix
import networkx as nx
# Use sparse representation for performance
sparse_sim = coo_matrix((weights, (row, col)), shape=similarity_matrix.shape)

# Initialize bipartite graph
B = nx.Graph()

# Add nodes for Platform A (0 to len(embeddings_a) - 1) and Platform B
B.add_nodes_from(range(len(embeddings1)), bipartite=0)
B.add_nodes_from(range(len(embeddings1), len(embeddings1) + len(embeddings2)), bipartite=1)

# Add edges from sparse similarity data
edges = [(r, len(embeddings1) + c, {"weight": w}) for r, c, w in zip(sparse_sim.row, sparse_sim.col, sparse_sim.data)]
B.add_edges_from(edges)

# Print graph information
print(f"Bipartite graph has {B.number_of_nodes()} nodes and {B.number_of_edges()} edges.")

Bipartite graph has 738139 nodes and 321504 edges.


In [28]:
components = list(nx.connected_components(B))

In [35]:
#find component larger than 1
components = [c for c in components if len(c) > 1]


In [32]:
print(f"Number of components larger than 1: {sum(np.array(num_components) > 1)}")

Number of components larger than 1: 1882


In [36]:
# Example: Retrieve posts for each component
platform_a_posts = list1
platform_b_posts = list2
component_posts = []
for component in components:
    posts = {
        "Platform A": [platform_a_posts[node] for node in component if node < len(platform_a_posts)],
        "Platform B": [platform_b_posts[node - len(platform_a_posts)] for node in component if node >= len(platform_a_posts)],
    }
    component_posts.append(posts)

In [ ]:
component_posts

In [38]:
from bertopic import BERTopic

# Combine posts from each component
all_posts = []
for posts in component_posts:
    all_posts.extend(posts["Platform A"] + posts["Platform B"])

# Fit BERTopic model
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(all_posts)

# Assign topics back to components
component_topics = []
index = 0
for posts in component_posts:
    num_posts = len(posts["Platform A"]) + len(posts["Platform B"])
    component_topics.append({
        "Component": posts,
        "Topics": topics[index:index + num_posts]
    })
    index += num_posts

# Print topics for the first component
print("Topics in first component:", component_topics[0]["Topics"])


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Topics in first component: [6, 129, 6, 135, 34, 128, 206, 323, 24, -1, 13, 76, 259, 461, 125, 55, 0, 179, 5, 82, 4, 36, 4, 461, -1, 28, 2, 205, -1, 0, 2, 4, 5, -1, 99, -1, 462, 5, 4, 130, -1, 55, 2, 133, 560, 4, 6, -1, 16, 433, 5, -1, 11, 2, 433, 5, -1, -1, -1, 0, 48, -1, 127, 127, 121, 78, -1, 247, 115, 105, 552, 21, 55, -1, -1, -1, 90, 4, -1, 533, 130, 47, 16, 130, 14, 28, 86, 209, 5, -1, 492, -1, 16, 104, -1, -1, -1, 0, -1, 508, 128, 468, 20, 15, -1, 168, 316, 15, -1, 76, 15, 532, 82, 511, 168, 536, 49, 187, 52, 52, 52, -1, 77, -1, 175, 175, 10, -1, 98, 0, 4, 535, -1, -1, -1, 82, 73, 0, 2, 47, 0, 98, 499, 4, 256, 0, 342, 273, 99, 125, 0, 104, 19, 130, -1, 58, 282, 93, -1, 0, -1, -1, -1, 0, 0, 0, 19, 0, 0, 203, -1, 468, 82, 105, 4, 111, 29, 289, 98, 47, 36, -1, 38, 10, 21, 6, 21, 6, 130, 290, 10, 2, 470, -1, 91, 16, 0, 16, 2, 2, -1, 93, -1, 15, 66, 36, 93, 130, 15, 20, 55, 5, 38, 38, 2, 2, -1, -1, 92, -1, -1, 2, 2, 2, 2, 65, -1, 2, 75, 456, 548, 10, -1, 123, 13, 2, 453, 47, 113, 23, 

In [45]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4776,-1_convicted_hunter_34_felon,"[convicted, hunter, 34, felon, found, is, coun...",[Biden was Biden tonight. Consistent with 50 p...
1,0,930,0_debate_first_presidential_off,"[debate, first, presidential, off, face, vs, d...",[What to watch for at Biden and Trump's first ...
2,1,810,1_n8_ichgehenichtmitdenkindernzulegoichgehenic...,"[n8, ichgehenichtmitdenkindernzulegoichgehenic...",[Ichgehenichtmitdenkindernzulegoichgehenichtmi...
3,2,341,2_felon_convicted_felonies_convict,"[felon, convicted, felonies, convict, bad, opp...",[Donald Trump Was Just Convicted Of 34 Felonie...
4,3,261,3_weapons_russia_strike_ukraine,"[weapons, russia, strike, ukraine, inside, use...",[Biden Administration Allows Ukraine To Use Am...
...,...,...,...,...,...
562,561,10,561_raspy_halting_simulcasting_restrained,"[raspy, halting, simulcasting, restrained, www...","[Biden starts debate raspy, stumbles through t..."
563,562,10,562_rule_google_learn_34,"[rule, google, learn, 34, felonies, activating...",[The former president has been convicted of 34...
564,563,10,563_doj_weaponized_weaponizing_excuse,"[doj, weaponized, weaponizing, excuse, operate...",[Is Hunter Biden Trial Just An Excuse For Dems...
565,564,10,564_gagged_returns_denies_gag,"[gagged, returns, denies, gag, appeal, order, ...",[BREAKING: New York Court Denies Trump Gag Ord...


In [46]:
data = []
# Iterate over components and map posts to their topics and IDs
for component in component_topics:
    # Process Platform A (bsky)
    for post, topic in zip(component["Component"]["Platform A"], component["Topics"][:len(component["Component"]["Platform A"])]):
        match_bsky = bsky_df[bsky_df["post"] == post]
        if not match_bsky.empty:
            data.append({"id": match_bsky["id"].values[0], "post": post, "topic": topic})
    
    # Process Platform B (ts)
    for post, topic in zip(component["Component"]["Platform B"], component["Topics"][len(component["Component"]["Platform A"]):]):
        match_ts = ts_df[ts_df["post"] == post]
        if not match_ts.empty:
            data.append({"id": match_ts["id"].values[0], "post": post, "topic": topic})


In [59]:
import pandas as pd


# Collect all posts and their associated topics and component IDs
all_data = []
for component_id, component in enumerate(component_topics):  # Use enumerate to assign unique component_id
    # Add Platform A (bsky) posts
    all_data.extend([(component_id, post, topic) for post, topic in zip(
        component["Component"]["Platform A"], 
        component["Topics"][:len(component["Component"]["Platform A"])]
    )])
    
    # Add Platform B (ts) posts
    all_data.extend([(component_id, post, topic) for post, topic in zip(
        component["Component"]["Platform B"], 
        component["Topics"][len(component["Component"]["Platform A"]):]
    )])# Create a DataFrame for all posts with their component_id and topic
all_posts_df = pd.DataFrame(all_data, columns=["component_id", "post", "topic"])

# Merge with bsky_df
bsky_merged = all_posts_df.merge(bsky_df, on="post", how="inner")

# Merge with ts_df
ts_merged = all_posts_df.merge(ts_df, on="post", how="inner")

# Combine the results
export_df = pd.concat([bsky_merged, ts_merged])

# Reorder columns and export to CSV
export_df = export_df[["component_id", "id", "post", "topic"]]

In [56]:
export_df.to_csv("../data/matched_bsky_ts.csv", index=False)

In [72]:
export_df.loc[export_df['component_id'] == 34, ]

,component_id,id,post,topic
11541,34,at://did:plc:6nwn2kvqwfdbx64kl2benuv2/app.bsky...,[trump hanging himself on abortion]\n\nbiden:,101
11542,34,at://did:plc:sh4bxqvvnpzcyp2lkjhcz3od/app.bsky...,"Trump telling Biden ""Let's not act like childr...",-1
11543,34,at://did:plc:sq6w6264xndxwnfxp6be3pwr/app.bsky...,Biden and Trump spar on abortion and tax polic...,101
11544,34,at://did:plc:yx2nfpytmmze4fau2qxw2c5a/app.bsky...,Biden and Trump spar on abortion and tax polic...,101
11545,34,at://did:plc:4hm6gb7dzobynqrpypif3dck/app.bsky...,@NBCNews: The debate should have been a layup ...,101
11546,34,at://did:plc:sxk2txh376zqkdgbgmevaw6j/app.bsky...,BIDEN JUST CALLED TRUMP A CHILD!,-1
11547,34,at://did:plc:ieluoyrr7apgy6rw3uo5zk3v/app.bsky...,Biden’s anger over Trump lying about abortion ...,101
11548,34,at://did:plc:m7gu2vbtt2fid7xruraxnrr2/app.bsky...,"holy shit that is a wild own goal from Biden, ...",101
11549,34,at://did:plc:o4s55v3tsfph6whswxccpsia/app.bsky...,Biden finally pushes back on the right-wing mi...,101
11550,34,at://did:plc:xts2ui2jxnfpijz2o6oixska/app.bsky...,Biden do not fuck up this abortion question,101


In [47]:

# Create the DataFrame
export_df = pd.DataFrame(data)


In [49]:
export_df.to_csv("../data/matched_bsky_ts.csv", index=False)

In [51]:
# Save BERTopic model
topic_model.save("../models/bertopic_model")

# Export topic summary
topic_info = topic_model.get_topic_info()
topic_info.to_csv("../data/bertopic_topic_summary.csv", index=False)

# Extract and save top words for each topic
topics_dict = {}
for topic_id in topic_info["Topic"]:
    if topic_id != -1:  # Skip the "outlier" topic (-1)
        topics_dict[topic_id] = topic_model.get_topic(topic_id)

# Save topic keywords to a CSV
topics_keywords = []
for topic_id, keywords in topics_dict.items():
    keywords_str = ", ".join([f"{word} ({score:.2f})" for word, score in keywords])
    topics_keywords.append({"Topic": topic_id, "Keywords": keywords_str})

topics_df = pd.DataFrame(topics_keywords)
topics_df.to_csv("../data/bertopic_keywords.csv", index=False)


2024-11-21 18:51:45,417 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [53]:
export_df

,id,post,topic
0,at://did:plc:czjduztdqkrm6j2t5pik47s3/app.bsky...,Trump delivers remarks on his guilty verdict: ...,6
1,at://did:plc:frq374y4bvdwsvtaxzsrapcn/app.bsky...,Some Republicans less likely to vote for Trump...,129
2,at://did:plc:7xcqk4nimevs263cxjyjhope/app.bsky...,After the Trump verdict ...,6
3,at://did:plc:hyc74lpczraueyxu2mkryflk/app.bsky...,"holy crap, the U.S presidential debate is piti...",135
4,at://did:plc:flijdq6nzuscczuhvv2lt7bp/app.bsky...,Classified Doc Case: Prosecutors Renew Bid to ...,34
...,...,...,...
26449,112615006748638507,Massachusetts bill banning 'revenge porn' land...,12
26450,at://did:plc:wizueigbb4k7txa5bpnhcuk2/app.bsky...,Seriously?,-1
26451,112624383204427304,Seriously,-1
26452,at://did:plc:on5oeywiqx32fh2zau473wz6/app.bsky...,This sitting of the Supreme Court will stretch...,468


In [26]:
from tqdm.auto import tqdm
# Step 4: Compute cosine similarity
cosine_sim = cosine_similarity(embeddings1.cpu(), embeddings2.cpu())

# Step 5: Find matches based on similarity threshold
threshold = 0.75  # Adjust the threshold as needed
matches = []

import numpy as np

# Convert cosine_sim to a NumPy array (if it's not already one)
cosine_sim_array = np.array(cosine_sim)
print("Finished converting to NumPy array.")
# Find indices where the score exceeds the threshold
indices = np.argwhere(cosine_sim_array > threshold)
print("Finished computing matches.")
# Use the indices to build matches
matches = [(list1[i], list2[j], cosine_sim_array[i, j]) for i, j in indices]
print("Finished building matches.")


Finished converting to NumPy array.
Finished computing matches.
Finished building matches.


In [29]:
matches

[('Trump delivers remarks on his guilty verdict: Key moments and analysis apnews.com/live/trump-g...',
  'Quick message about the Trump verdict:',
  0.7501957),
 ('Trump delivers remarks on his guilty verdict: Key moments and analysis apnews.com/live/trump-g...',
  'Response To Trump Guilty Verdict',
  0.7660806),
 ('Trump delivers remarks on his guilty verdict: Key moments and analysis apnews.com/live/trump-g...',
  "Reacting To President Trump's Guilty Verdict",
  0.79584706),
 ('Some Republicans less likely to vote for Trump after guilty verdict, poll finds',
  'Trump verdict makes significant number of Republicans less likely to support him: poll',
  0.79945767),
 ('Some Republicans less likely to vote for Trump after guilty verdict, poll finds',
  'MORE MAGA THAN EVER: Guilty Verdict Makes Half of Republicans More Likely to Vote for Trump; Poll',
  0.766623),
 ('Some Republicans less likely to vote for Trump after guilty verdict, poll finds',
  'Trump verdict makes significant num

In [30]:
ts_df.loc[ts_df['post']=="MORE MAGA THAN EVER: Guilty Verdict Makes Half of Republicans More Likely to Vote for Trump; Poll",]

,id,post
10100,112536868532387285,MORE MAGA THAN EVER: Guilty Verdict Makes Half...
